# 如何并行调用 runnable

:::info 先决条件

本指南假定您熟悉以下概念：
- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [链接 runnables](/docs/how_to/sequence)

:::

[`RunnableParallel`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableParallel.html) 原始类型本质上是一个 dict，其值是 runnables（或可以被强制转换为 runnables 的东西，例如函数）。它并行运行所有值，并且每个值都使用 `RunnableParallel` 的整体输入进行调用。最终的返回值是一个 dict，其中包含每个值在其对应键下的结果。

## 使用 `RunnableParallels` 进行格式化

`RunnableParallels` 可用于并行化操作，也可用于将一个 Runnable 的输出格式化以匹配序列中下一个 Runnable 的输入格式。您可以使用它们来拆分或分叉链，以便多个组件可以并行处理输入。之后，其他组件可以合并结果来综合最终响应。这种类型的链创建了一个如下所示的计算图：

```text
     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
      Combine
```

下面，prompt 的输入预期是一个具有 `"context"` 和 `"question"` 键的 map。用户输入只是问题。因此，我们需要使用我们的检索器来获取上下文，并将用户输入作为 `"question"` 键传递。

In [ ]:
# | output: false
# | echo: false

%pip install -qU langchain langchain_openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# The prompt expects input with keys for "context" and "question"
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

:::tip
请注意，当我们将 `RunnableParallel` 与另一个 `Runnable` 组合时，我们甚至不需要将字典再次包装在 `RunnableParallel` 类中 —— 类型转换已为我们处理。在链的上下文中，以下是等效的：
:::

```
{"context": retriever, "question": RunnablePassthrough()}
```

```
RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
```

```
RunnableParallel(context=retriever, question=RunnablePassthrough())
```

请参阅关于[强制类型转换的章节](/docs/how_to/sequence/#coercion)。

## 使用 itemgetter 作为简写

请注意，在使用 `RunnableParallel` 时，您可以使用 Python 的 `itemgetter` 作为简写来从映射中提取数据。您可以在 [Python 文档](https://docs.python.org/3/library/operator.html#operator.itemgetter) 中找到关于 itemgetter 的更多信息。

在下面的示例中，我们使用 itemgetter 从映射中提取特定键：

In [3]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "where did harrison work", "language": "italian"})

'Harrison ha lavorato a Kensho.'

## 并行化步骤

RunnableParallels 可以轻松地并行执行多个 Runnables，并将这些 Runnables 的输出作为 Map 返回。

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content="Why don't bears like fast food? Because they can't catch it!", response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 13, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe024170-c251-4b7a-bfd4-64a3737c67f2-0'),
 'poem': AIMessage(content='In the quiet of the forest, the bear roams free\nMajestic and wild, a sight to see.', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 15, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-2707913e-a743-4101-b6ec-840df4568a76-0')}

## 并行

RunnableParallel 也可用于并行运行独立的进程，因为映射中的每个 Runnable 都会并行执行。例如，我们可以看到我们之前的 `joke_chain`、`poem_chain` 和 `map_chain` 的运行时间都差不多，尽管 `map_chain` 同时执行了其他 দুটো。

In [5]:
%%timeit

joke_chain.invoke({"topic": "bear"})

610 ms ± 64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit

poem_chain.invoke({"topic": "bear"})

599 ms ± 73.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit

map_chain.invoke({"topic": "bear"})

643 ms ± 77.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 后续步骤

现在您已经了解了一些使用 `RunnableParallel` 来格式化和并行化链式操作的方法。

如需了解更多信息，请参阅本节中关于可运行组件的其他操作指南。